#### libs

In [6]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np

import pymeshlab
import polyscope as ps
import open3d as o3d

#### Diffeomorphic registrations

##### feed in SCHREC19

In [2]:
# 1) get starting .obj and ending .obj as numpy arrays
# 2) feed into net
# 3) visually reconstruct object using polyscope, reconstructing normals and meshes using open3d

# 1 - feeding correctly formatted data into the neural net. This is optimised to the SCHREC19 dataset.
def convert_obj_to_numpy(obj):
    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(obj)
    m = ms.current_mesh()
    v_matrix = m.vertex_matrix()
    return v_matrix

def collect_source_and_target(sourcefile, targetfile):
    source_shape  = convert_obj_to_numpy(sourcefile)
    target_shape  = convert_obj_to_numpy(targetfile)

    source_shape = source_shape.reshape((source_shape.shape[0], 1, 3))
    target_shape = target_shape.reshape((target_shape.shape[0], 1, 3))

    source_scaling = np.max(source_shape)
    source_bias    = np.min(source_shape)
    source_shape   = (source_shape - np.min(source_shape))/np.max(source_shape)
    source_shape   = (source_shape - np.min(source_shape))/np.max(source_shape)


    target_scaling = np.max(target_shape)
    target_bias    = np.min(target_shape)
    target_shape   = (target_shape - np.min(target_shape))/np.max(target_shape)
    target_shape   = (target_shape - np.min(target_shape))/np.max(target_shape)

    source_shape = tf.convert_to_tensor(source_shape)
    target_shape = tf.convert_to_tensor(target_shape)
    
    return source_shape, target_shape

# source_shape, target_shape = collect_source_and_target('scan_015.obj', 'scan_016.obj')

##### initialise and train diffeomorphic net

In [22]:
# 2 - initialising the diffeomorphic net

width = 800
batch_size=source_shape.shape[0]

class DenseEulerFBlock(tf.keras.Model):
    def __init__(self):
        super(DenseEulerFBlock, self).__init__()
        self.initialiser = tf.keras.initializers.GlorotUniform()
        
        self.d1 = tf.keras.layers.Dense(width, activation='relu')
        self.d2 = tf.keras.layers.Dense(width, activation=None)
        self.d3 = tf.keras.layers.Dense(3, activation=None, use_bias=False)
        
    def call(self, input_tensor, training=False):
        return self.d3(self.d2(self.d1(input_tensor)))


class DenseEulerMergeBlock(tf.keras.Model):
    def __init__(self):
        super(DenseEulerMergeBlock, self).__init__()
        
    def call(self, input_tensor, training=False):
        return tf.nn.relu(input_tensor)


def cd_so(y_pred, y_true):
    cd1 =  tf.math.reduce_sum(tf.math.sqrt(
           tf.math.reduce_min(tf.math.reduce_sum(
           tf.math.square(tf.reshape(y_pred, (batch_size, 1, 1, 3)) - y_true), axis=-1), axis=1)))
    cd2 =  tf.math.reduce_sum(tf.math.sqrt(
           tf.math.reduce_min(tf.math.reduce_sum(
           tf.math.square(tf.reshape(y_true, (batch_size, 1, 1, 3)) - y_pred), axis=-1), axis=1)))
    return cd1 + cd2


def DenseCombinedCDLoss(d1, d2, d3, d4, d5, d6, m6, truth, sigma=0.1):
    regularisation_loss = 0.5*(tf.norm(d1) + tf.norm(d2) + tf.norm(d3) + tf.norm(d4) + tf.norm(d5) + tf.norm(d6))/6
    data_term = 0.5*cd_so(m6, truth)/(sigma**2)
    return data_term + regularisation_loss


tf.keras.backend.clear_session()
input0 = tf.keras.Input(shape=(1, 3))

d1 = DenseEulerFBlock()(input0)
m1 = DenseEulerMergeBlock()(input0 + d1)

d2 = DenseEulerFBlock()(m1)
m2 = DenseEulerMergeBlock()(m1 + d2)

d3 = DenseEulerFBlock()(m2)
m3 = DenseEulerMergeBlock()(m2 + d3)

d4 = DenseEulerFBlock()(m3)
m4 = DenseEulerMergeBlock()(m3 + d4)

d5 = DenseEulerFBlock()(m4)
m5 = DenseEulerMergeBlock()(m4 + d5)

d6 = DenseEulerFBlock()(m5)
m6 = DenseEulerMergeBlock()(m5 + d6)


true0 = tf.keras.Input(shape=(1, 3))
model = tf.keras.Model([input0, true0], [input0, m1, m2, m3, m4, m5, m6, true0])
model.add_loss(DenseCombinedCDLoss(d1, d2, d3, d4, d5, d6, m6, true0, sigma=0.1)) # hoping this works? (I haven't done EMD or chamfer's distance)

opt = tf.keras.optimizers.Adam(learning_rate=5e-6,beta_1=0.9,beta_2=0.999, epsilon=1e-07)
model.compile(optimizer=opt, loss=None)

# 2 - running the net
batch_size=source_shape.shape[0]
model.fit(x=[source_shape, target_shape], y=None, epochs=1000, verbose=1, batch_size=batch_size);
plt.plot(model.history.history['loss']);
print(model.history.history['loss'][-1])

##### reconstruct flow

In [4]:
# 3 - construction

def reconstruct_numpy_to_points_and_vertices(np_arr):
    ms = pymeshlab.Mesh(np_arr)
    ms.generate_surface_reconstruction_ball_pivoting()
    m = ms.current_mesh()
    v_matrix = m.vertex_matrix()
    f_matrix = m.face_matrix()
    return [v_matrix, f_matrix]


def make_pointed_flow(model, source_shape, target_shape, separator=50):
    prediction = model.predict([source_shape, target_shape])
    prediction[0] = prediction[0].reshape((prediction[0].shape[0], 3))
    full_flow = np.copy(prediction[0]*100)
    for i in range(1, len(prediction)):
        prediction[i] = prediction[i].reshape((prediction[i].shape[0], 3))
        full_flow = np.append(full_flow, prediction[i]*100 + i*separator*np.array([1, 0, 0]), axis=0)
    return full_flow


# recommended...
# saving the diffeomorphism for huge reductions in stress

# np.save('full_flow', visualise_flow(model, source_shape, target_shape))
# point_cloud = np.load('full_flow.npy')


# point_cloud = np.load('full_flow.npy')
def visualise_flow(flow_array, pts_per_obj=None, view_style='turntable', mesh_type='ball', mesh=True):
    ps.init()
    ps.set_navigation_style(view_style)
    if mesh is not True:
        # visualise non-meshed flow (just a point-cloud)  
        ps.register_point_cloud("pointed_flow", point_cloud)

    elif mesh is True:
        if pts_per_obj is None:
            raise ValueError("As you are meshing, you need to specify `pts_per_obj`. Generally this is flow_array.shape[0]/(resnet_lddmm timesteps + 2)")
            
        for i in range(flow_array.shape[0]//pts_per_obj):
            # visualising it meshed
            obj = flow_array[pts_per_obj*i:pts_per_obj*(i+1)]

            # import points into the open3d 03d object
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(obj)
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=5, max_nn=100)) # estimates normals for the point cloud
            
            if mesh_type == 'ball':
                radius = np.mean(pcd.compute_nearest_neighbor_distance())
                meshes = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius]))
            
            elif mesh_type == 'poisson':
                # computes the smooth poisson mesh of the point cloud
                meshes = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=11, width=0, scale=1.5, linear_fit=True)[0]
            
            elif mesh_type == 'alpha':
                alpha_val = 0.1 # adjust as necessary
                meshes = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha_val)

            # write mesh to .obj file, which will then be viewed by pymeshlab and then polyscope
            o3d.io.write_triangle_mesh("full_flow_meshed_pivoted" + "{}".format(i) + ".obj", meshes)

            # create pymeshlab object that will then have states and properties stored
            ms = pymeshlab.MeshSet()
            ms.load_new_mesh('full_flow_meshed_pivoted' + '{}'.format(i) + '.obj')
            m = ms.current_mesh()

            # get numpy arrays of vertices and faces of the current mesh
            v_matrix = m.vertex_matrix()
            f_matrix = m.face_matrix()


            # visualise with polyscope
            # a=ps.register_point_cloud("full_flow_rasterised {}".format(i), v_matrix)
            b=ps.register_surface_mesh("full_flow_meshed {}".format(i), v_matrix, f_matrix, smooth_shade=True)
            b.set_back_face_policy('identical')
    ps.show()

# visualise_meshed_flow(point_cloud, 1001)